In [1]:
import numpy as np
from utils import pelt, apart
from pathlib import Path
import time

In [2]:
mean_vec = [2, 4, 1, 5]

def const_changes(N):
    return np.repeat(mean_vec, N // len(mean_vec))

def linear_changes(N):
    return np.tile(np.repeat(mean_vec, 10), N // (len(mean_vec)*10))

sim_fun_dict = {
    'const_changes': const_changes,
    'linear_changes': linear_changes
}

method_dict = {
    'pelt': pelt,
    'apart': apart
}

In [3]:
# Hyperparamters
d = 1       # Dimensionality
s = 10      # Number of discrete value for each dimension
trials = 4  # Number of trials per signal length
n = 18      # 2^{n} is the max length of trial signal
lda = 10    # penalty

# Discrete means set
angles = np.linspace(0, 2 * np.pi, s, endpoint=False)
grids = np.meshgrid(*([angles] * d), indexing='ij')
Theta = np.stack(grids, axis=-1).reshape(-1, d)

In [ ]:
# setting up recorded csv
for method in ['pelt', 'apart']:
    for signal_type in ['const_changes', 'linear_changes']:
        output_path = Path(f"benchmark_results/{method}_time_vs_samples_{signal_type}_{d}D_{s}MeansGrid_penalty{lda}.csv")

        # Write CSV header once
        if not output_path.exists():
            with open(output_path, 'w') as f:
                f.write("signal_length,mean_exe_time_sec,std_exe_time_sec\n")

        for i in range(6, n+1):
            signal_length = 2 ** i
            exe_times = np.empty(trials, dtype=np.float64)
            for j in range(trials):
                # Generate random signal
                signal = np.column_stack([
                    np.clip(np.random.normal(loc=sim_fun_dict[signal_type](signal_length), scale=0.4), 0, 2 * np.pi)
                    for _ in range(d)
                ])

                # Time the function
                start = time.perf_counter()
                chpnts, signal_mean = method_dict[method](signal, Theta, lda)
                end = time.perf_counter()

                exe_times[j] = end - start

            # Compute stats
            mean_time = np.mean(exe_times)
            std_time  = np.std(exe_times)

            # Record to CSV
            with open(output_path, 'a') as f:
                f.write(f"{signal_length},{mean_time:.6f},{std_time:.6f}\n")